## Load Dataset

In [1]:
!git clone https://github.com/Shujun-Wu-98/Catheter-and-Line.git

fatal: destination path 'Catheter-and-Line' already exists and is not an empty directory.


In [2]:
!pip install opencv-python
!pip install timm
!pip install -U git+https://github.com/albu/albumentations

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/comm

## Libraries

In [1]:
import numpy as np
import cv2
from PIL import Image
import time
import pandas as pd
import timm
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import albumentations
from albumentations import *
from albumentations.pytorch import ToTensorV2

In [2]:
#device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:

TEST_IMAGE_DIR = '/content/Catheter-and-Line/test/'
sub_dir = '/content/Catheter-and-Line/sample_submission.csv'

BATCH_SIZE = 8
IMAGE_SIZE = 512

df_test = pd.read_csv(sub_dir)
LABELS = [
            'ETT - Abnormal',
            'ETT - Borderline',
            'ETT - Normal',
            'NGT - Abnormal',
            'NGT - Borderline',
            'NGT - Incompletely Imaged',
            'NGT - Normal',
            'CVC - Abnormal',
            'CVC - Borderline',
            'CVC - Normal',
            'Swan Ganz Catheter Present'
]
df_test.head()

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present
0,1.2.826.0.1.3680043.8.498.46923145579096002617...,0,0,0,0,0,0,0,0,0,0,0
1,1.2.826.0.1.3680043.8.498.84006870182611080091...,0,0,0,0,0,0,0,0,0,0,0
2,1.2.826.0.1.3680043.8.498.12219033294413119947...,0,0,0,0,0,0,0,0,0,0,0
3,1.2.826.0.1.3680043.8.498.84994474380235968109...,0,0,0,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.8.498.35798987793805669662...,0,0,0,0,0,0,0,0,0,0,0


## Dataset and Model

In [4]:
#load dataset
class Ranzcr_Images(Dataset):

    def __init__(self, df, transform=None):
        
        self.df = df
        self.image_ids = df["StudyInstanceUID"].values
        self.transform = transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, index):
        """ Returns image,labels. """

        image_id = self.image_ids[index]
        path = TEST_IMAGE_DIR + str(image_id) + ".jpg"
        image = cv2.imread(path,0)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform is not None:
            res = self.transform(image=image)
            img = res['image']

        return img, image_id

In [5]:
#define model
class RanzcrNet(nn.Module):
  def __init__(self, model_name='resnet200d', out_dim = 11):
    super().__init__()
    self.model = timm.create_model(model_name, pretrained=True)
    n_features = self.model.fc.in_features
    self.model.global_pool = nn.Identity()
    self.model.fc = nn.Identity()
    self.pooling = nn.AdaptiveAvgPool2d(1)
    self.fc = nn.Linear(n_features, out_dim)


  def forward(self, x):
    bs = x.size(0)
    features = self.model(x)
    pooled_features = self.pooling(features).view(bs,-1)
    output = self.fc(pooled_features)
    
    return output

## Transform

In [6]:
transforms_test = albumentations.Compose([
    Resize(IMAGE_SIZE,IMAGE_SIZE),
    Normalize(                    #using ImageNet norms
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    ToTensorV2()
])

In [7]:
test_dataset = Ranzcr_Images(
    df_test, 
    transform=transforms_test,
    )

test_loader = DataLoader(
    test_dataset,
    batch_size = 64,
    shuffle = False,
    num_workers = 2,
)

## Util

In [8]:
def fzto(pred):
  for i in range(len(pred)):
    for j in range(len(pred[i])):
      if pred[i][j]<0.5:
        pred[i][j]=0
      else:
        pred[i][j]=1
  return pred

## Predicted

In [9]:
model = RanzcrNet()
model = nn.DataParallel(model)

model.load_state_dict(torch.load('/content/model9.pkl'))
model.eval()

model.to(device)

DataParallel(
  (module): RanzcrNet(
    (model): ResNet(
      (conv1): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, 

In [10]:
p = torch.zeros(1,11)
id_total = []
with torch.no_grad():
  for index,data in enumerate(test_loader):
    print(index)
    image, ids = data[0].to(device),data[1]
    for id in ids:
      id_total.append(id)
    prediction = model(image)
    pred = prediction.sigmoid().detach().cpu()
    result = fzto(pred)
    p = torch.cat((p,result),0)
_,p = torch.split(p,[1,len(test_dataset)])

0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


In [11]:
print(p.shape)
print(len(id_total))

torch.Size([3582, 11])
3582


In [13]:
print(p[0:5])

tensor([[0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])


In [12]:
outputs = {
    'StudyInstanceUID': id_total,
    'ETT - Abnormal': p[:,0],
    'ETT - Borderline': p[:,1],
    'ETT - Normal': p[:,2],
    'NGT - Abnormal': p[:,3],
    'NGT - Borderline': p[:,4],
    'NGT - Incompletely Imaged': p[:,5],
    'NGT - Normal': p[:,6],
    'CVC - Abnormal': p[:,7],
    'CVC - Borderline': p[:,8],
    'CVC - Normal': p[:,9],
    'Swan Ganz Catheter Present': p[:,10]
}

dataframe = pd.DataFrame(outputs, columns=['StudyInstanceUID','ETT - Abnormal','ETT - Borderline','ETT - Normal',
                                           'NGT - Abnormal','NGT - Borderline','NGT - Incompletely Imaged','NGT - Normal',
                                           'CVC - Abnormal','CVC - Borderline','CVC - Normal','Swan Ganz Catheter Present'])
dataframe.to_csv('output.csv',index=False)